# Projeto 2 - NLP

-----

Nome:  
Turma:

Os segundo projeto do módulo de Dados não estruturados II será focado no processamento de linguagem natural! Usaremos os algoritmos aprendidos e as técnicas vistas na segunda parte do curso para extrairmos informações relevantes de texto. Mais precisamente, de publicações no Twitter.

## Os Dados

Utilizaremos um Dataset obtido do Twitter com 100K postagens entre os dias 01/08/2018 e 20/10/2018. Cada postagem é classificada como **positiva**, **negativa** ou **neutra**.  

Dois arquivos serão disponilizados para o desenvolvimento dos modelos, um para treino/validação e outro para submissão. Os arquivos se encontram na pasta */Dados/train* e */Dados/subm*, respectivamente.

Descrição das colunas:

- **id**: ID único para o tweet  
- **tweet_text**: Texto da publicação no Twitter  
- **tweet_date**: Data da publicação no Twitter  
- **sentiment**: 0, se negativo; 1, se positivo; 2, se neutro  
- **query_used**: Filtro utilizado para buscar a publicação

## O Problema

Você deverá desenvolver um modelo para detectar o sentimento de uma publicação do Twitter a classificando em uma das três categorias: **positiva**, **negativa** ou **neutra**. O texto da publicação está disponível na coluna "tweet_text". Teste pelo menos 2 técnicas de NLP diferentes e escolha a métrica de avaliação que julgar mais pertinente.  

Escolha o melhor modelo e gere uma base a partir dos dados de submissão, que estão no caminho ```Dados/subm/Subm3Classes.csv```, com o seguinte formato:


|id|sentiment_predict
|-|-|
|12123232|0
|323212|1
|342235|2

Salve essa tabela como um arquivo csv com o nome ```<nome>_<sobrenome>_nlp_degree.csv``` e submeta-o como parte da entrega final do projeto.  

Para ajudar no desenvolvimento, é possível dividir o projeto em algumas fases:

- **Análise de consistência dos dados**: analise se os dados estão fazendo sentido, se os campos estão completos e se há dados duplicados ou faltantes. Se julgar necessário, trate-os.    


- **Análise exploratória**: analise a sua base como um todo, verifique o balanceamento entre as classes e foque, principalmente, na coluna ```tweet_text```.    


- **Pré-processamento e transformações**: projetos de NLP exigem um considerável pré-processamento. Foque no tratamento da string do texto. Procure começar com tratamentos simples e adicione complexidade gradualmente. Nessa etapa você testará diferentes técnicas de transformações, como o Bag Of Words e o TF-IDF.    


- **Treinamento do modelo**: depois das transformações, você poderá executar o treinamento do modelo classificador. Nessa etapa o problema se torna semelhante aos abordados nos módulos anteriores. Você pode testar diversos classificadores como RandomForest, AdaBoost, entre outros. Otimize os hiperparâmetros do modelo com técnicas como a GridSearch e a RandomizedSearch.    


- **Conclusões**: descreva, em texto, as conclusões sobre os seus estudos. O modelo é capaz de identificar o sentimento das publicações? É possível extrapolar o modelo para outros contextos, como a análise de sentimento de uma frase qualquer? Pense em questões pertinentes e relevantes que você tenha obtido durante o desenvolvimento do projeto!     



## Critérios de avaliação

Os seguintes itens serão avaliados:

1. Desenvolvimento das etapas descritas acima;


2. Reprodutibilidade do código: seu código será executado e precisa gerar os mesmos resultados apresentados por você;


3. Clareza: seu código precisa ser claro e deve existir uma linha de raciocínio direta. Comente o código em pontos que julgar necessário para o entendimento total;


4. Justificativa das conclusões obitdas: não existirá certo ou errado, mas as decisões e as conclusões precisam ser bem justificadas com base nos resultados obtidos.  

O desempenho do modelo **não** será considerado como critério de avaliação.  

## Informações gerais

- O projeto deve ser desenvolvido individualmente ou em grupo;



- Entrega através do Class: Processamento Digital de Imagens - Definições e Fundamentos
 -> Exercícios -> Projeto 2


Anexar, na entrega, o notebook de desenvolvimento e o arquivo .csv de submissão, da seguinte forma:  

Criar um arquivo zip com:
- notebook: ```<nome>_<sobrenome>_<númeroTurma>_projeto_2.ipynb```   
- csv: ```<nome>_<sobrenome>_<númeroTurma>_projeto_2_submissao.csv```


## Dicas

### Base de treino e submissão

A base de submissão não possui a variável de saída, portanto ela será utilizada **apenas** para gerar o arquivo que acompanha a submissão do projeto.      

### Tente encontrar possíveis vieses

É muito comum que modelos de NLP possuam fortes vieses, como a tendência de relacionar palavras específicas com alguma classe de saída. Tente encontrar vieses no seu estudo, isso pode ajudar a tirar boas conclusões. o campo "query_used" pode ser útil para essa análise.  

### O pré-processamento é a chave para um bom desempenho

Essa é a etapa que mais vai contribuir para o desempenho do seu modelo. Seja criativo e desenvolva essa etapa de uma maneira que seja fácil de aplicar o mesmo processamento para uma nova base, você terá que fazer isso para gerar a base de submissão.

-------

# Bibliotecas

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import re

pio.renderers
pio.renderers.default = "notebook_connected"

# Análise de consistência dos dados

- Não foram encontrados dados nulos;
- Foram encontrados dados duplicados porque esses dados são resultados de diferentes queries;
- As queries possuem alto víes. Cada query possui apenas um sentimento relacionado.

## Leitura dos dados

In [2]:
training_file_path = "Dados/train/Train3Classes.csv"

df = pd.read_csv(training_file_path)

df.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1049721159292346368,Rio elege maior bancada policial de sua histór...,Tue Oct 09 18:00:01 +0000 2018,2,folha
1,1046251157025423360,fiquei tão triste quando eu vi o preço da câme...,Sun Sep 30 04:11:28 +0000 2018,0,:(
2,1041744620206653440,"Para Theresa May, seu plano para o Brexit é a ...",Mon Sep 17 17:44:06 +0000 2018,2,exame
3,1046937084727107589,caralho eu quero proteger a danielly em um pot...,Tue Oct 02 01:37:06 +0000 2018,0,:(
4,1047326854229778432,@SiCaetano_ viva o caos :),Wed Oct 03 03:25:55 +0000 2018,1,:)


In [3]:
df.columns

Index(['id', 'tweet_text', 'tweet_date', 'sentiment', 'query_used'], dtype='object')

## Verifica se tem valores nulos

Não foram encontrados dados nulos.

In [4]:
df.isnull().sum()

id            0
tweet_text    0
tweet_date    0
sentiment     0
query_used    0
dtype: int64

## Verifica valores duplicados

Dados duplicados foram encontrados porque foram buscados por queries diferentes.

In [5]:
for column in df.columns:
    exists_duplicated_values = df[column].duplicated().sum()
    print(f"{column}: {exists_duplicated_values}")

id: 13
tweet_text: 816
tweet_date: 11713
sentiment: 94997
query_used: 94986


In [6]:
df["id"].duplicated().sum()

13

In [7]:
duplicated_ids = df[df["id"].duplicated()]["id"]

df.query("id in @duplicated_ids").sort_values("id")

,id,tweet_text,tweet_date,sentiment,query_used
39184,1037082879438729216,O que é #FATO ou #FAKE na entrevista de Gerald...,Tue Sep 04 21:00:01 +0000 2018,2,#fato
38913,1037082879438729216,O que é #FATO ou #FAKE na entrevista de Gerald...,Tue Sep 04 21:00:01 +0000 2018,2,jornaloglobo
35012,1037837855433928704,Veja o que é #FATO ou #FAKE na entrevista de E...,Thu Sep 06 23:00:01 +0000 2018,2,#fato
20226,1037837855433928704,Veja o que é #FATO ou #FAKE na entrevista de E...,Thu Sep 06 23:00:01 +0000 2018,2,jornaloglobo
39324,1038222887432273920,Veja o que é #FATO ou #FAKE na entrevista de F...,Sat Sep 08 00:30:00 +0000 2018,2,jornaloglobo
83565,1038222887432273920,Veja o que é #FATO ou #FAKE na entrevista de F...,Sat Sep 08 00:30:00 +0000 2018,2,#fato
69100,1038494427503837185,Veja o que é #FATO ou #FAKE na entrevista de H...,Sat Sep 08 18:29:00 +0000 2018,2,#fato
35849,1038494427503837185,Veja o que é #FATO ou #FAKE na entrevista de H...,Sat Sep 08 18:29:00 +0000 2018,2,jornaloglobo
62236,1038570177191993344,Veja o que é #FATO ou #FAKE na entrevista de H...,Sat Sep 08 23:30:00 +0000 2018,2,#fato
93665,1038570177191993344,Veja o que é #FATO ou #FAKE na entrevista de H...,Sat Sep 08 23:30:00 +0000 2018,2,jornaloglobo


## Análise dos dados pela query

As queries possuem alto víes. Cada query possui apenas um sentimento relacionado.

In [8]:
queries = df["query_used"].unique()

queries

array(['folha', ':(', 'exame', ':)', '#fato', 'g1', '#novidade',
       '#noticia', 'estadao', 'jornaloglobo', '#curiosidade',
       '#oportunidade', 'veja', '#trabalho'], dtype=object)

In [9]:
sentiment_by_queries = []

for query in queries:
    df_by_query = df[df["query_used"] == query]

    print("=" * 20)
    print(f"Query: {query}")

    sentiment_counted = df_by_query['sentiment'].value_counts()
    sentiment_counted_dict = sentiment_counted.to_dict()

    for (key, value) in list(sentiment_counted_dict.items()):
        print(f"Sentiment: {key} | Counted: {value}")

        sentiment_by_query = {
            "sentiment": key,
            "counted": value,
            "query": query,
        }

        sentiment_by_queries.append(sentiment_by_query)

Query: folha
Sentiment: 2 | Counted: 5004
Query: :(
Sentiment: 0 | Counted: 31696
Query: exame
Sentiment: 2 | Counted: 3417
Query: :)
Sentiment: 1 | Counted: 31678
Query: #fato
Sentiment: 2 | Counted: 3471
Query: g1
Sentiment: 2 | Counted: 3439
Query: #novidade
Sentiment: 2 | Counted: 920
Query: #noticia
Sentiment: 2 | Counted: 1114
Query: estadao
Sentiment: 2 | Counted: 3880
Query: jornaloglobo
Sentiment: 2 | Counted: 2374
Query: #curiosidade
Sentiment: 2 | Counted: 381
Query: #oportunidade
Sentiment: 2 | Counted: 2455
Query: veja
Sentiment: 2 | Counted: 2141
Query: #trabalho
Sentiment: 2 | Counted: 3030


In [10]:
pd.DataFrame(sentiment_by_queries)

,sentiment,counted,query
0,2,5004,folha
1,0,31696,:(
2,2,3417,exame
3,1,31678,:)
4,2,3471,#fato
5,2,3439,g1
6,2,920,#novidade
7,2,1114,#noticia
8,2,3880,estadao
9,2,2374,jornaloglobo


# Análise exploratória

## Pré-processamento para análise exploratória

In [11]:
_df = df.copy()

def update_data(row, total):
    row["tweet_text_length"] = len(row["tweet_text"])
    row["tweet_time"] = re.search("\d{2}:\d{2}:\d{2}", row["tweet_date"]).group(0)
    
    hour = int(row["tweet_time"][:2])
    if hour < 6:
        row["tweet_period"] = "Dawn"
    elif hour < 12:
        row["tweet_period"] = "Morning"
    elif hour < 18:
        row["tweet_period"] = "Afternoon"
    else:
        row["tweet_period"] = "Night"

    if row["sentiment"] == 0:
        row["sentiment_name"] = "Negative"
    elif row["sentiment"] == 1:
        row["sentiment_name"] = "Positive"
    else:
        row["sentiment_name"] = "Neutral"

    if row.name % 500 == 0:
        value = (row.name + 1) / total * 100
        print(f"Progress: {value:.2f}%")
    
    return row

total = len(_df)

_df = _df.apply(
    lambda row: update_data(row, total),
    axis=1
)

print("Progress: 100.00%")

Progress: 0.00%
Progress: 0.53%
Progress: 1.05%
Progress: 1.58%
Progress: 2.11%
Progress: 2.63%
Progress: 3.16%
Progress: 3.69%
Progress: 4.21%
Progress: 4.74%
Progress: 5.26%
Progress: 5.79%
Progress: 6.32%
Progress: 6.84%
Progress: 7.37%
Progress: 7.90%
Progress: 8.42%
Progress: 8.95%
Progress: 9.47%
Progress: 10.00%
Progress: 10.53%
Progress: 11.05%
Progress: 11.58%
Progress: 12.11%
Progress: 12.63%
Progress: 13.16%
Progress: 13.69%
Progress: 14.21%
Progress: 14.74%
Progress: 15.26%
Progress: 15.79%
Progress: 16.32%
Progress: 16.84%
Progress: 17.37%
Progress: 17.90%
Progress: 18.42%
Progress: 18.95%
Progress: 19.47%
Progress: 20.00%
Progress: 20.53%
Progress: 21.05%
Progress: 21.58%
Progress: 22.11%
Progress: 22.63%
Progress: 23.16%
Progress: 23.69%
Progress: 24.21%
Progress: 24.74%
Progress: 25.26%
Progress: 25.79%
Progress: 26.32%
Progress: 26.84%
Progress: 27.37%
Progress: 27.90%
Progress: 28.42%
Progress: 28.95%
Progress: 29.47%
Progress: 30.00%
Progress: 30.53%
Progress: 31.05%

## Verifica tamanho do campo ```tweet_text```

Verificando o tamanho do campo ```tweet_text``` observa-se que tweets com maior tamanho em média são neutros, os tweets com menor tamanho em média são negativos e os tweets positivos ficaram com tamanho intermediário.

In [12]:
data = _df.groupby(by="sentiment_name").mean().reset_index()[["sentiment_name", "tweet_text_length"]]

fig = px.line(data, x="sentiment_name", y="tweet_text_length", title='Sentiment by tweet text length')

fig.update_xaxes(title="Sentiment")
fig.update_yaxes(title="Tweet text length")

fig.show()

## Verificando a hora do tweet

Verificando o período de cada tweet, foi possível verificar a maioria dos tweet no período do madrugada são negativos. Nos outros períodos, os sentimentos se distribuem de forma mais homogênea.

In [13]:
data = _df[["sentiment_name", "tweet_period"]]
data = data.groupby(by=["sentiment_name", "tweet_period"]).size().reset_index()
data.columns = ["sentiment_name", "tweet_period", "total"]

fig = px.bar(
    data,
    x="sentiment_name",
    y="total",
    color="tweet_period",
    title='Sentiment by tweet period',
    barmode='group',
)

fig.update_xaxes(title="Sentiment")
fig.update_yaxes(title="Tweet period")

fig.show()

## Verificar balanceamento

In [14]:
_df.columns

Index(['id', 'tweet_text', 'tweet_date', 'sentiment', 'query_used',
       'tweet_text_length', 'tweet_time', 'tweet_period', 'sentiment_name'],
      dtype='object')

In [15]:
_df.groupby("sentiment")[[
    'sentiment', 'query_used',
       'tweet_text_length', 'tweet_time', 'tweet_period', 'sentiment_name']]

In [16]:
_df.cumsum()

: 

: 

# Pré-processamento e transformações

# Treinamento do modelo

# Conclusões